In [10]:
from src import CONST
import numpy as np
import cv2
import os
import time
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras import applications
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd

In [11]:
# PREPROCESSING DATASET -----------------
print("Loading dataset...")

#grab the paths to our input images followed by shuffling them 
imagePaths = sorted(list(paths.list_images('dataset')))

Loading dataset...


In [12]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('dataset'):
    if not j.startswith('.'): # If running this code locally, this is to 
        print(j)                      # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

MX4
Peace
fist


{'MX4': 0, 'Peace': 1, 'fist': 2}

In [14]:
data = []
labels = []
lookup = {}
for imagePath in imagePaths:
    # load the image, pre-process it, resize it to IMAGE_SIZE, store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (CONST.IMAGE_SIZE, CONST.IMAGE_SIZE))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    folders = []
    for folder in os.listdir('dataset'):
        if not folder.startswith('.'):
            folders.append(folder)
    for i in range(len(folders)):
        if label == folders[i]:
            label = i
            labels.append(label)
            if folders[i] not in lookup.keys():
                lookup[i]= folders[i]
            else:
                pass
    
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [15]:
# scaling the data points from [0, 255] to the range [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data,labels,test_size = 0.1)
X_train[0].shape

(100, 100, 1)

In [17]:
# convert the labels from integers to vectors
NUM_CLASSES = len(folders)

y_train = to_categorical(y_train).astype(int)
y_test = to_categorical(y_test).astype(int)

In [18]:
def save_bottlebeck_features():
    
    print("Training the VGG16 bottleneck features ...")
    
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights=None, input_shape=(100,100,1)) 
    
    bottleneck_features_train = model.predict(X_train, verbose=1)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)
    
    bottleneck_features_validation  = model.predict(X_test, verbose=1)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

In [19]:
save_bottlebeck_features()

Training the VGG16 bottleneck features ...
Instructions for updating:
Colocations handled automatically by placer.
30/30 [==============================] - 3s 115ms/step


In [20]:
print("[INFO] training the top of the model...")

[INFO] training the top of the model...


In [21]:
#save_bottlebeck_features()

train_data = np.load('bottleneck_features_train.npy')
validation_data = np.load('bottleneck_features_validation.npy')


In [22]:
from keras import layers

In [23]:
top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.6))
top_model.add(Dense(3, activation='sigmoid'))

In [24]:
top_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
EPOCHS = 50
BATCH_SIZE = 2
NUM_CLASSES = 3
TEST_SIZE = 0.25

In [ ]:
start = time.time()

model_info = top_model.fit(train_data, y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,
                           validation_data=(validation_data, y_test),
                           verbose=1)
end = time.time()

print ("\nModel training time: %0.1fs\n" % (end - start))

Instructions for updating:
Use tf.cast instead.
Train on 270 samples, validate on 30 samples
Epoch 1/50
270/270 [==============================] - 3s 12ms/step - loss: 1.0991 - accuracy: 0.3444 - val_loss: 1.0998 - val_accuracy: 0.2000
Epoch 2/50
270/270 [==============================] - 2s 7ms/step - loss: 1.0987 - accuracy: 0.3519 - val_loss: 1.1001 - val_accuracy: 0.2000
Epoch 3/50
270/270 [==============================] - 2s 6ms/step - loss: 1.0979 - accuracy: 0.3519 - val_loss: 1.0997 - val_accuracy: 0.2000
Epoch 4/50
270/270 [==============================] - 2s 6ms/step - loss: 1.0973 - accuracy: 0.3667 - val_loss: 1.1002 - val_accuracy: 0.2000
Epoch 5/50
270/270 [==============================] - 2s 6ms/step - loss: 1.0953 - accuracy: 0.3963 - val_loss: 1.1002 - val_accuracy: 0.2000
Epoch 6/50
270/270 [==============================] - 2s 6ms/step - loss: 1.0966 - accuracy: 0.3778 - val_loss: 1.0978 - val_accuracy: 0.2000
Epoch 7/50
270/270 [==============================] - 

In [43]:
scores = top_model.evaluate(validation_data, y_test)
print("\nTest Loss:  %.2f%%" % (scores[0]*100))
print("Test Accuracy: %.2f%%\n" % (scores[1]*100))

30/30 [==============================] - 0s 248us/step

Test Loss:  111.19%
Test Accuracy: 20.00%



In [ ]:
# Saving model
'''if not os.path.isdir(CONST.SAVE_DIR):
    os.makedirs(CONST.SAVE_DIR)
model_path = os.path.join(CONST.SAVE_DIR, CONST.BOTTLENECK_MODEL)
top_model.save(model_path)
print('\nSaved trained model at %s ' % model_path)'''